So if we want the value after 0, let's see if it looks as though there's a pattern.

In [1]:
import itertools as it
import pandas as pd

In [2]:
def spinlock_generator(stepSizeIn_i, iterations_i):
    '''
    Return a generator with step size stepSizeIn_i, that
    has run for iterations_i cycles.
    '''

    spinlock_cyc=it.cycle([0])
    for n in range(iterations_i):
        
        # Step forward the given number of steps:
        for i in range(stepSizeIn_i+1):
            next(spinlock_cyc)

        newCycle_ls=[n+1]+list(it.islice(spinlock_cyc, 0, n+1))

        spinlock_cyc=it.cycle(newCycle_ls)
        
    return spinlock_cyc
        

Write a quick function to get the value after zero:

In [3]:
def value_after_zero(iteratorIn_cyc):
    '''Run through an iterator.cycle object, and return
       the first value found after 0
    '''
    while not next(iteratorIn_cyc)==0:
        pass
    return next(iteratorIn_cyc)

In [4]:
assert value_after_zero(spinlock_generator(3, 4))==2

For the test input we get:

In [5]:
# [value_after_zero(spinlock_generator(3, i)) for i in range(1000)]

# Edited out to avoid a massive output

Nothing obvious there, I don't think.

Perhaps what we want to be doing is looking at when a number is added when 0 is at the end of the cycle. So where is 0 after each cycle?

1. [1,0] $p=1$
2. [2,1,0] $p=2$
3. [3,1,0,2] $p=2$
4. [4,3,1,0,2] $p=3$
5. [5,2,4,3,1,0] $p=6$
6. [6,1,0,5,2,4,3] $p=3$
7. [7,2,4,3,6,1,0,5] $p=6$


This will be easier to visualise as a `pd.Series`:

In [6]:
pd.Series([list(it.islice(spinlock_generator(3, i), i+1)).index(0) for i in range(10)])

0    0
1    1
2    2
3    2
4    3
5    5
6    2
7    6
8    3
9    9
dtype: int64

OK, so let's say we have i (the index), and f(i) is the position of the 0:
* If f(i)==step size, then f(i+1)==i+1, and the value following 0 is i+1
* if f(i)>step size, then f(i+1)==f(i)-step size
* if f(i)<step size, then f(i+1)=i+f(i)-step size+1



Now, this behaviour is obviously slightly different in the opening stages, so in the interests of speed/laziness, I'm just going to calculate the $n$th case. I'm not proud.

In [7]:
stepSize_i=3

i=stepSize_i
fi=list(it.islice(spinlock_generator(stepSize_i, i), i+1)).index(0)

valueAfter0_i=fi

while i<20:
    print('{}, {}'.format(i, fi))
    if fi==stepSize_i:
        fi=i+1
        valueAfter0_i=i+1
    elif fi>stepSize_i:
        fi=fi-stepSize_i
    elif fi<stepSize_i:
        fi=i+fi-stepSize_i+1
    i+=1
    
valueAfter0_i

3, 2
4, 3
5, 5
6, 2
7, 6
8, 3
9, 9
10, 6
11, 3
12, 12
13, 9
14, 6
15, 3
16, 16
17, 13
18, 10
19, 7


16

Good, that seems to work. So let's find the fifty millionth for my puzzle input:

In [8]:
stepSize_i=363

i=stepSize_i
fi=list(it.islice(spinlock_generator(stepSize_i, i), i+1)).index(0)

valueAfter0_i=fi

while i<50000000:

    if fi==stepSize_i:
        fi=i+1
        valueAfter0_i=i+1
    elif fi>stepSize_i:
        fi=fi-stepSize_i
    elif fi<stepSize_i:
        fi=i+fi-stepSize_i+1
    i+=1

valueAfter0_i

1080289